<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Telco Network Coverage Analysis
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial'>
Mobile communication is an essential technology in the present era. Telecommunication towers which support mobile telecommunication services are constantly constructed as the community need increases. To outline when and where the new towers need to be built, users can utilize Teradata Vantage and ClearScape Analytics. Vantage’s capabilities aid in determining the number of mobile subscribers and traffic capacity in each area, using geospatial analysis which incorporates geographical and demand data. Once, the telco network and connectivity issues are fully analyzed, telco companies can create the appropriate number of base stations and telecommunication towers.
</p>

<p style = 'font-size:18px;font-family:Arial'><b>Business Values</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Driving Spatial Insights for Network Optimization to:
        <ul style = 'font-size:16px;font-family:Arial'>
            <li>Optimize network layouts</li>
            <li>Minimize construction costs</li>
            <li>Plan least-cost network reducing capital expenditures</li></ul>
    </li>
    <li>Calculate the number of mobile subscribers and traffic capacity</li>
    <li>Determine the number of base stations and telecommunication towers</li>
    <li>Customer acquisition: Determine the service users who will subscribe through Geographic Information Systems (GIS)</li>
 </ul>
<p style = 'font-size:18px;font-family:Arial'><b>Why Vantage?</b></p>  
<p style = 'font-size:16px;font-family:Arial'>The benefit of geospatial analysis in Vantage starts with the ability to combine this type of analytics with traditional and advanced ML techniques at scale. With these capabilities, users can create deeper and more accurate insights. Although there are a variety of Geographic Information Systems (GIS) tools available, Vantage has unique advantages. 
Driving actionable insights from location-based data requires a platform scalable enough to analyze data at the volume of entire populations. In order to drive real-world insights, this platform needs to enable both traditional and advanced ML and AI techniques. ClearScape Analytics combines these analytic disciplines into a single, massively scalable platform which enables unique business outcomes and more accurate analytic and predictive models.
</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>1. Connect to Vantage</b></p>

<p style = 'font-size:16px;font-family:Arial'>Let's start by importing the libraries needed.</p>

In [ ]:
# Standard libraries
import getpass
import warnings

# Third-party libraries
import geopandas as gpd
import matplotlib.pyplot as plt 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import json
from pandas import json_normalize
from  ipywidgets import widgets, interact

# Teradata libraries
from teradataml import *
display.max_rows = 5

# Suppress warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import plotly.io as pio
pio.renderers.default = 'notebook'  


<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql("SET query_band='DEMO=Telco_Network_Coverage_PY_SQL.ipynb;' UPDATE FOR SESSION;")

<p style = 'font-size:18px;font-family:Arial'> <b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>

In [ ]:
 #%run -i ../run_procedure.py "call get_data('DEMO_TelcoNetwork_cloud');"
 # takes about 30 seconds, estimated space: 0 MB
%run -i ../run_procedure.py "call get_data('DEMO_TelcoNetwork_local');" 
# takes about 30 seconds, estimated space: 3 MB

<p style = 'font-size:16px;font-family:Arial'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>2. Initial Data Sets</b></p>

<p style = 'font-size:16px;font-family:Arial'>Let us start by looking at the data we have and what information we can derive from those.</p>
<p style = 'font-size:18px;font-family:Arial'><b>2.1 Cell Tower</b></p>
<p style = 'font-size:16px;font-family:Arial'>Geographical location (Longitude and Latitude) of Cell Towers with additional details of the cell towers were given in this table. Let's see how the data looks like.<br>
The teradataml GeoDataFrame module function creates a pointer to a table in the target Vantage system. Note, no data is copied back to the client, and all operations on the data are translated to SQL and executed in Vantage. We use GeoDataFrame when we have any geometry datatype like Point, Linestring etc in our table otherwise we use teradataml DataFrame. </p>

In [ ]:
res1 = GeoDataFrame(in_schema("DEMO_TelcoNetwork", "Cell_Towers"))
res1

<p style = 'font-size:16px;font-family:Arial'>Let's check the datatypes of the columns.</p>

In [ ]:
res1.tdtypes

<p style = 'font-size:16px;font-family:Arial'>From above we can see that cell_geom column is of Geometry datatype.<br>We can also see that the table has details of the cell towers. Let's visualize this data to better understand the location of towers on the map.<br>Clearscape Analytics offers seamless integration with third-party Geo visualization tools such as QGIS or various Python modules like Plotly. The approach involves conducting all calculations and pre-processing within Vantage itself. This way, only essential information will be passed on to the visualization tools. This not only accelerates the computation process but also minimizes the time spent on data movement between different tools. Ultimately, this strategy enhances efficiency in both calculation and visualization phases of the data analysis.</p>

In [ ]:
#cell tower sites
res1.drop(['radio_type','radio_freq','degree_facing','cell_id','postal_area_code','postal_area_name','dist_kms_from_map_centroid','cell_geom'], axis=1)
cell_tower = res1[(res1.cell_lon < 152)].to_pandas()

In [ ]:
fig = px.scatter_mapbox(cell_tower, lat="cell_lat", lon="cell_lon", hover_name="cell_site_name", 
                        hover_data=["site_num"],
                        color_discrete_sequence=["black"], zoom=7, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<p style = 'font-size:16px;font-family:Arial'>Here we can see the tower locations on the map.</p> 

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.2 Aggregated Customer Experience</b></p>
<p style = 'font-size:16px;font-family:Arial'>Geographical points (Longitude and Latitude) of customer experience using RSCP/signal dBm values were given in this table. Let's see how the data looks like in this table.</p>

In [ ]:
res2 = GeoDataFrame(in_schema("DEMO_TelcoNetwork", "Cust_Experience"))
res2

In [ ]:
#dbm point values
nw = res2.assign(drop_columns=True,
                rscpavg = res2.RSCPAVG,
                lat = res2.POINT.y,
                lon = res2.POINT.x,
                point = res2.POINT,
                signal = case([(res2.RSCPAVG <= -101, 'poor'),
                               (res2.RSCPAVG <= -86, 'good'),
                               (res2.RSCPAVG >= -85, 'excellent') ],
                         else_='na') 
               )                            
                              

<p style = 'font-size:16px;font-family:Arial'>
We have found that the coverage area of a BTS antenna represents the distribution of signals across the Earth's surface. Several factors come into play when determining this coverage area, including the antenna type, environmental conditions, propagation model, cell radius, and power budget. It's essential to consider these aspects as they collectively influence the extent and effectiveness of the signal coverage from the antenna.</p>
<p style = 'font-size:16px;font-family:Arial'>
We measure the signal strength in RSCP (Received Signal Code Power), representing the power detected on a specific physical communication channel. This measurement serves as an indicator of signal strength and plays a crucial role as a criterion for handover decisions.<br>We have categorized the signal strength in poor, good and excellent based on the dBm values.</p>
<p style = 'font-size:16px;font-family:Arial'>Let's plot the information in the map for better understanding.</p>

In [ ]:
nw=nw.to_pandas(all_rows=True)
fig = px.scatter_mapbox(nw, lat="lat", lon="lon",  
                        hover_data=["rscpavg"],
                        color="signal"  ,
                        color_discrete_map={
                                             "poor": "red",
                                             "good": "yellow",
                                             "excellent": "green",
                                             "na": "black"
                                          } ,
                        labels={"signal": "rscpavg" },
                        zoom=11, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

<p style = 'font-size:16px;font-family:Arial'>In this visualization we have divided our signal strengths into categories of excellent (rscpavg greater than -85) /good (rscpavg between -100 and -80) /poor (rscpavg less than -100) based on the rscpavg value we have received from the corresponding location on Vantage and passed only the categories for visualization.</p>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.3 Coveage Map</b></p>
<p style = 'font-size:16px;font-family:Arial'>We were provided with a GeoJSON file of a geographical area, and this file covers the network area under consideration.</p>


In [ ]:
with open ("./data/main_map.geojson",'r') as infile:
    counties = json.load(infile)

In [ ]:
#for colorcoding different coverage area
df1 = json_normalize(counties['features'])
c_code = df1["properties.SITEID"].unique()
df2 = pd.DataFrame(c_code)
df3 = []
for i in range(len(c_code)):
    df3.append([c_code[i],i+1])
df = pd.DataFrame(df3, columns = ['siteid', 'colourcode'])


In [ ]:
fig = px.choropleth_mapbox(df, geojson=counties,
                           locations='siteid',
                           featureidkey='properties.SITEID',
                           color='colourcode',
                           hover_name='siteid',
                           range_color=(0,30),
                           mapbox_style="open-street-map",
                           color_continuous_scale="Viridis",
                           zoom=10,height=300,center ={"lat":-33.06363, "lon":151.74148},
                           opacity=0.5
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


<p style = 'font-size:16px;font-family:Arial'>In the visualization we can see the area marked on the map. We can also load this GeoJson file in Vantage using <b>GeoImport Utility</b></p>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>2.4 Customer Navigation</b></p>
<p style = 'font-size:16px;font-family:Arial'>Customer Navigation table contains the customer navigation steps for 364 journeys. This serves as the superset data, encompassing crucial details such as the incremental journey sequence, timestamp for each step, signal strength values (signal_dBm), geographical coordinates for each journey, mobile app information, throughput, and more. Let's explore the information in this table for a more in-depth analysis. </p>

In [ ]:
res3 = GeoDataFrame(in_schema("DEMO_TelcoNetwork", "Cust_Journey"))
res3

In [ ]:
res3.tdtypes

<p style = 'font-size:16px;font-family:Arial'>From this table we can create 1 row for each customer journey trajectory using <b>ClearScape Analytics UAF(unbounded array framework) </b>. UAF is the Vantage framework for building end-to-end time series forecasting pipelines. For further information on UAF, please <a href = 'https://docs.teradata.com/search/all?query=Unbounded+Array+Framework+(UAF)&content-lang=en-US '>refer here</a> <br><br> <b>TRACKINGOP</b> is a multi-dimensional function for geospatial data in UAF which can calculate trip distance, speed, time for a trip. It helps users to optimize their resources by providing insights into the movements of their assets. For example, by tracking the movement of a fleet of vehicles, a logistics company can identify the most efficient routes, optimize delivery schedules, and reduce fuel consumption. In our usecase it gives information about the routes/locations that customers use often and signal strengths they received etc. Let's calculate this based on our data.</p>    

<p style = 'font-size:16px;font-family:Arial'>Let us create a subset of data of few customers.</p>

In [ ]:
res4 = res3[res3.customer_id.isin(['CID-0005005','CID-0005073','CID-0005233','CID-0005243'])]

In [ ]:
from sqlalchemy import literal_column
column = literal_column("journey_dttm + interval '1' second", type_=res4.journey_dttm.type)

In [ ]:
column2 = literal_column("cast(journey_geom as SYSUDTLIB.ST_GEOMETRY(400))")

In [ ]:
uaf_customer = res4.assign(drop_columns=True,
               customer_id = res4.customer_id,
               journey_id  = res4.journey_id,
               journey_seq = res4.journey_seq,            
               signal_dBm = res4.signal_dBm,
               arv_ts = res4.journey_dttm,
               dep_ts = column,
               geom= column2,
               journey_geom = res4.journey_geom)            

In [ ]:
uaf_customer

<p style = 'font-size:16px;font-family:Arial'>Let us check data for a single customer.</p>

In [ ]:
uaf_customer[(uaf_customer.customer_id == 'CID-0005243')]

<p style = 'font-size:16px;font-family:Arial'>Let's create a uaf series from our data which we can input to the TrackingOp function.</p>

In [ ]:
# Create teradataml TDSeries object.
uaf_series_df = TDSeries(data=uaf_customer,
                              id=["customer_id", "journey_id"],
                              row_index="arv_ts",
                              row_index_style="TIMECODE",
                              payload_field=["arv_ts", "dep_ts", "geom"],
                              payload_content="MULTIVAR_ANYTYPE")

<p style = 'font-size:16px;font-family:Arial'>Now let's use the TrackingOp function to calculate the distance, time, speed metrics for our customer routes.</p>

In [ ]:
uaf_out = TrackingOp(data=uaf_series_df,
                         distance=True,
                         speed=True,
                         time_spent=True,
                         metric=True)

<p style = 'font-size:16px;font-family:Arial'>Let's check the output of function for the customer CID-0005243</p>

In [ ]:
uaf_res = uaf_out.result
uaf_res[(uaf_res.customer_id == 'CID-0005243')]

<p style = 'font-size:16px;font-family:Arial'>From the above output, we can see the customer trajectory along with his speed and total trip time and trip distance. We can also plot the customer trajectory for better visualization. Again for this we will pass only the essential information and data-preparation calculations will be done on Vantage itself.</p>

In [ ]:
#customer navigation path
nw_path2 = res4.assign(drop_columns=True,
                customer_id = res4.customer_id,
                journey_id = res4.journey_id,
                journey_seq = res4.journey_seq,
                signal_dBm = res4.signal_dBm,    
                lat = res4.journey_geom.y,
                lon = res4.journey_geom.x,
                color_scale = case([(res4.signal_dBm <= -101, 0), #poor
                               (res4.signal_dBm <= -86, 1),  #good
                               (res4.signal_dBm >= -85, 2) ], #excellent
                         else_= 0) #na
               )  
nw_path2.sort(['customer_id','journey_id','journey_seq'])
nw_path2=nw_path2.to_pandas(all_rows=True).reset_index()

In [ ]:
def plot_map(customer):    
    
    custom_colors = [
    [0, '#FF0000'], # red
    [0.5, '#FFEA00'], # yellow
    [1, '#028A0F'] # green
    ]
    
    plt.figure(figsize=(8, 6))
    df = nw_path2[nw_path2['customer_id'] == customer ]
    fig = px.line_mapbox(df, lat="lat", lon="lon",
                     hover_data=["journey_id", "customer_id"],
                     color="journey_id"  ,
                     zoom=8, height=300)
    fig.add_trace(go.Scattermapbox(
    name = "signal",
    mode = "markers",
    lat = df.lat.tolist(),
    lon = df.lon.tolist(),
    hoverinfo='text',
    hovertemplate= ['<b>Signal_dBm:</b>:' + str(df.iloc[i, 3]) for i in range(df.shape[0])],
    marker_color=df['color_scale'] ,
    marker=dict(
        color=[1, 2, 3],
        colorscale=custom_colors,
        size = 8
    )    

))

    fig.update_layout(mapbox_style="open-street-map" , #"stamen-terrain",
                  mapbox_zoom=9,
                  mapbox_center_lon=151.5,
                  mapbox_center_lat=-33.1,
                  margin={"r":0,"t":0,"l":0,"b":0}
                 )
    
       
    return fig

# Create the dropdown widget
customer = ["CID-0005005","CID-0005073", "CID-0005233","CID-0005243"]
customer_dropdown = widgets.Dropdown(options=customer, description='Customer:', value='CID-0005005')

# Call the plot_clusters function with the selected dropdown options
def update_plot(customer):    
    plot_map(customer).show()
    
widgets.interact(update_plot, customer=customer_dropdown)

<p style = 'font-size:16px;font-family:Arial'>Please use the dropdown to see the navigation paths for customer. We have also marked the signal strength at points where data was captured based on the bDm values.</p>

<p style = 'font-size:20px;font-family:Arial'><b>Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial'>In this demonstration, we have seen that we can effortlessly perform complex geospatial calculations in Vantage. The results can be seamlessly exported to visualization tools. The analysis of service area maps holds significant potential as it serves as a valuable reference for stakeholders, aiding them in making crucial decisions regarding telecommunications development and business expansion.<br>The distribution map of towers in digital form emerges as a practical tool, enabling efficient monitoring and organization of BTS assets owned by the company in a map view. This digital representation enhances operational field capabilities, contributing to a more streamlined management of the company's assets.</p>

<hr style="height:2px;border:none;">
<p style = 'font-size:20px;font-family:Arial'><b>3. Cleanup</b></p>

<p style = 'font-size:16px;font-family:Arial'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_TelcoNetwork');" 
#Takes 10 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;">


<b style = 'font-size:20px;font-family:Arial'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:18px;font-family:Arial'><b>Filters:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
<li><b>Industry:</b> Telco</li>
<li><b>Functionality:</b> Geospatial</li>
<li><b>Use Case:</b> Mobile Data Geographic Analysis</li>
</ul>
<p style = 'font-size:18px;font-family:Arial'><b>Related Resources:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
<li><a href = 'https://www.teradata.com/Blogs/Digital-utilities-driving-value-from-analytics'>Digital utilities driving value from advanced analytics at scale</a></li>
<li><a href = 'https://www.teradata.com/Blogs/Hyper-scale-time-series-forecasting-done-right'>Hyper-scale time series forecasting done right</a></li>
<li><a href = 'https://www.teradata.com/Industries/Energy-and-Natural-Resources'>Gain Analytic Insights for the Energy Industry</a></li>
    </ul>

<p style = 'font-size:18px;font-family:Arial'><b>Reference Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'> 
       <li>Teradata Vantage™ - Unbounded Array Framework Time Series Reference: <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference/Unbounded-Array-Framework '>https://docs.teradata.com/r/Teradata-VantageTM-Unbounded-Array-Framework-Time-Series-Reference/Unbounded-Array-Framework </a></li>    
  <li>Teradata® Geospatial Utilities User Guide: <a href = 'https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview/Welcome-to-Teradata-Tools-and-Utilities-Teradata-Geospatial-Utilities-User-Guide'>https://docs.teradata.com/r/Teradata-Geospatial-Utilities-User-Guide/June-2022/Teradata-Geospatial-Utilities-Overview/Welcome-to-Teradata-Tools-and-Utilities-Teradata-Geospatial-Utilities-User-Guide</a></li>
  <li>Python Pandas Reference: <a href = 'https://pandas.pydata.org/docs/user_guide/index.html'>https://pandas.pydata.org/docs/user_guide/index.html</a></li>
<li>Plotly Reference: <a href = 'https://plotly.com/'>https://plotly.com/</a></li>
      
</ul>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023,2024. All Rights Reserved
        </div>
    </div>
</footer>